In [11]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import scipy.special
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')
from sklearn.model_selection import train_test_split
import os

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score

from sklearn import metrics
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import TomekLinks


from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE

##### from collections import Counter
from sklearn.datasets import make_classification
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

from imblearn.ensemble import EasyEnsembleClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


import scipy.stats as st
import math


#Import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder


import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pandas as pd
seed=45


## Importing Training Set

In [12]:
data=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/training_set.csv')
data=data.iloc[:,1:]
data.head()

X = data.drop(columns='CIMEMLOS')
Y = data['CIMEMLOS']

## Importing Validation Set

In [13]:
eval_set=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/evaluation_set.csv')
eval_set=eval_set.iloc[:,1:]
eval_set.head()
print(eval_set.shape)

eval_X = eval_set.drop(columns='CIMEMLOS')
eval_Y = eval_set['CIMEMLOS']

(2401, 18)


## Creation of Cost Matrix

In [14]:
# Change FP, FN costs for different cost matrices.
# FP Cost: if col == 0 , FN cost: if col == 1 
# Suggested values FP: FN = 1.5, 9.5 and 1.5, 8.31

rows, cols = (len(data), 4)
arr = [[0]*cols]*rows
for row in range(0,rows):
    for col in range(cols):
        if col==0:
            arr[row][col]=1.5
        if col==1:
            arr[row][col]=10
#print(arr)
cost_matrix = pd.DataFrame(arr)  
print(cost_matrix)

         0   1  2  3
0      1.5  10  0  0
1      1.5  10  0  0
2      1.5  10  0  0
3      1.5  10  0  0
4      1.5  10  0  0
...    ...  .. .. ..
16804  1.5  10  0  0
16805  1.5  10  0  0
16806  1.5  10  0  0
16807  1.5  10  0  0
16808  1.5  10  0  0

[16809 rows x 4 columns]


## concating the cost matrix with Training Set

In [15]:
dataframe3=pd.concat([data, cost_matrix], axis=1)
dataframe3.head()
print(len(dataframe3))


16809


## Segregating X, Y, Cost Matrix for training

In [16]:
X = dataframe3.drop(['CIMEMLOS', 0,1,2,3], axis=1)
Y=dataframe3['CIMEMLOS']
print(Y.head())

Z=dataframe3[[0,1,2,3]]
print(Z.head())

0    0
1    0
2    0
3    0
4    0
Name: CIMEMLOS, dtype: int64
     0   1  2  3
0  1.5  10  0  0
1  1.5  10  0  0
2  1.5  10  0  0
3  1.5  10  0  0
4  1.5  10  0  0


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from costcla.datasets import load_creditscoring1
from costcla.models import CostSensitiveBaggingClassifier
from costcla.metrics import savings_score


## Model Fitting

In [18]:
f = CostSensitiveBaggingClassifier(n_estimators=10)
X=X.to_numpy()
Y=Y.to_numpy()
Z=Z.to_numpy()

f.fit(X, Y, Z )

CostSensitiveBaggingClassifier()

## Model Evaluation on the Evaluation Set

In [19]:
eval_X=eval_X.to_numpy()
y_pred=f.predict(eval_X)

In [20]:
print(confusion_matrix(eval_Y, y_pred))
print(classification_report(eval_Y, y_pred))
y_pred_proba=f.predict_proba(eval_X)[::,1]

    # Calculation of AUC
auc=metrics.roc_auc_score(eval_Y, y_pred_proba)
print(auc)

[[1713  407]
 [ 146  135]]
              precision    recall  f1-score   support

           0       0.92      0.81      0.86      2120
           1       0.25      0.48      0.33       281

    accuracy                           0.77      2401
   macro avg       0.59      0.64      0.59      2401
weighted avg       0.84      0.77      0.80      2401

0.687519304371181


## combining the Training and Validation Set

In [21]:
train_eval=[data, eval_set]
complete_train=pd.concat(train_eval)



In [22]:
complete_train.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2391, 2392, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2400],
           dtype='int64', length=19210)

In [23]:
# remove own index with default index
complete_train.reset_index(inplace = True, drop = True)

In [24]:
complete_train

,CIMEMLOS,SEXVAR,X.AGEG5YR,X.BMI5,EXERANY2,SLEPTIM1,ADDEPEV3,DIABETE4,CVDSTRK3,smok_everyday,smok_somday,smok_no,SES,White,Black,Hispanic,Multi,Other
0,0,0,10,28.29,1,7,0,1,0,0,0,1,11,1,0,0,0,0
1,0,1,11,37.97,0,6,0,1,0,0,0,1,5,1,0,0,0,0
2,0,0,9,19.84,1,8,0,0,0,1,0,0,14,1,0,0,0,0
3,0,0,7,21.11,0,4,1,1,0,0,0,1,6,1,0,0,0,0
4,0,1,11,20.53,0,4,0,0,0,1,0,0,12,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19205,0,0,9,31.57,1,9,0,0,0,0,0,1,6,0,0,1,0,0
19206,0,1,13,26.93,0,8,0,0,0,0,0,1,5,0,0,1,0,0
19207,0,1,10,34.46,0,6,0,1,0,0,0,1,3,0,0,1,0,0
19208,0,1,8,27.12,1,8,0,0,0,0,0,1,8,0,0,1,0,0


## Re creation of new cost matrix

In [25]:
rows, cols = (len(complete_train), 4)
arr = [[0]*cols]*rows
for row in range(0,rows):
    for col in range(cols):
        if col==0:
            arr[row][col]=1.5
        if col==1:
            arr[row][col]=10
#print(arr)
cost_matrix = pd.DataFrame(arr)  
print(cost_matrix)




         0   1  2  3
0      1.5  10  0  0
1      1.5  10  0  0
2      1.5  10  0  0
3      1.5  10  0  0
4      1.5  10  0  0
...    ...  .. .. ..
19205  1.5  10  0  0
19206  1.5  10  0  0
19207  1.5  10  0  0
19208  1.5  10  0  0
19209  1.5  10  0  0

[19210 rows x 4 columns]


## Concatinating the Complete Training Set and Cost Matrix

In [26]:
d4=pd.concat([complete_train, cost_matrix], axis=1)
d4.head()
print(len(d4))



19210


## Preparing X, Y, Z for final training in the classifer

In [27]:
X = d4.drop(['CIMEMLOS', 0,1,2,3], axis=1)
Y=d4['CIMEMLOS']
print(Y.head())

Z=d4[[0,1,2,3]]
print(Z.head())

0    0
1    0
2    0
3    0
4    0
Name: CIMEMLOS, dtype: int64
     0   1  2  3
0  1.5  10  0  0
1  1.5  10  0  0
2  1.5  10  0  0
3  1.5  10  0  0
4  1.5  10  0  0


In [28]:
len(X)

19210

## Final Model Fitting

In [29]:
f = CostSensitiveBaggingClassifier(n_estimators=10)
X=X.to_numpy()
Y=Y.to_numpy()
Z=Z.to_numpy()

f.fit(X, Y, Z )

CostSensitiveBaggingClassifier()

## Final Testing

In [30]:
test_data=pd.read_csv('/Users/ibrain/Downloads/Cognitive Neuroscience Work/Cognition and Lifestyle R Coding/Cognition_Lifestyle/Imbalanced Data/testing_set.csv')
test_data=test_data.iloc[:,1:]
test_data.head()
print(test_data.shape)

test_X = test_data.drop(columns='CIMEMLOS')
test_Y = test_data['CIMEMLOS']

(4802, 18)


In [31]:
test_X=test_X.to_numpy()
y_pred=f.predict(test_X)

In [32]:
print(confusion_matrix(test_Y, y_pred))
print(classification_report(test_Y, y_pred))
y_pred_proba=f.predict_proba(test_X)[:,1]

    # Calculation of AUC
auc=metrics.roc_auc_score(test_Y, y_pred_proba)
print(auc)

[[3449  848]
 [ 267  238]]
              precision    recall  f1-score   support

           0       0.93      0.80      0.86      4297
           1       0.22      0.47      0.30       505

    accuracy                           0.77      4802
   macro avg       0.57      0.64      0.58      4802
weighted avg       0.85      0.77      0.80      4802

0.6810436477671504
